In [28]:
#!/usr/bin/env python -W ignore::DeprecationWarning


import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
import xgboost
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler 
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn import preprocessing
import graphviz 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
# Stacking
from vecstack import stacking
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
import pyltr
import random as random
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings('ignore')

    
%matplotlib inline
import seaborn as sns
sns.set(context = 'paper', palette = 'winter_r', style = 'darkgrid', rc= {'figure.facecolor': 'gray',}, font_scale=1.5)


def grid(model,parameters):
    grid = GridSearchCV(estimator = model, param_grid = parameters, cv = 3, 
                        scoring = 'roc_auc')
    grid.fit(npX,npy_binary.ravel()) #npy
    return grid.best_score_, grid.best_estimator_.get_params()

def imp_features(model, model_name, params):
    Model = model(**params)
    Model.fit(npX,npy)
    names = X.columns
    feature = Model.feature_importances_
   
    important_features = pd.Series(data = feature, index = names,)
    important_features = important_features.sort_values(ascending = True)
    return important_features.plot(kind = 'barh', grid = False,title = model_name)

def f_selection(model,model_name, params, n):
    
    Model = model(**params)
    Model.fit(npX,npy)
    names = X.columns
    feature = Model.feature_importances_
    #selector = RFECV(Model, step=1, cv=15)
    selector = RFE(Model, n, step=1)
    selector = selector.fit(npX,npy)
    
    important_features = pd.Series(data = feature, index = names,)
    important_features = important_features.sort_values(ascending = False)
    return important_features, names, selector.get_support(indices=True)

def testing_confusion_phase(y_test,test):
    
    lb = preprocessing.LabelBinarizer()
    ytest_binary=lb.fit_transform(y_test)


# predicted values
    lb = preprocessing.LabelBinarizer()
    test_binary=lb.fit_transform(test)


    #print('The roc_auc score is:',roc_auc_score(ytest_binary, test_binary))


    cm = confusion_matrix(ytest_binary, test_binary)

    TP, TN, FP, FN = cm[1,1], cm[0, 0], cm[0,1], cm[1,0]
    #print("Outcomes on test data: \n", 
     # "Sensitivity/Recall = {:.3%}".format(TP/(TP+FN)), '\n',
    # "Specificity = {:.3%}".format(TN/(TN+FP)), '\n',
     #"Accuracy = {:.3%}".format((TP+TN)/(TP+TN+FP+FN))) # Validated against output of sklearn metrics' accuracy_score class


#     sns.heatmap(cm, annot=True, fmt='.0f', xticklabels=["Lost", "Won"], 
#            yticklabels=["Lost", "Won"])
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.title('Confusion matrix for test data')
#     plt.show()
    
    Sensitivity_Recall=TP/(TP+FN);
    Specificity=TN/(TN+FP)
    Accuracy=(TP+TN)/(TP+TN+FP+FN)
    
    return Sensitivity_Recall, Specificity, Accuracy
    
clf_rf = RandomForestClassifier() #RandomForest
clf_et = ExtraTreesClassifier() #ExtraTrees
clf_bc = BaggingClassifier() #Bagging
clf_ada = AdaBoostClassifier() #AdaBoost
clf_dt = DecisionTreeClassifier() #DecisionTree
clf_xg = XGBClassifier() #XGBoost
clf_lr = LogisticRegression() #LogisticRegression
clf_svm = SVC() #SVM
clf_nnet = MLPClassifier() # nnet

model_rf = RandomForestClassifier #RandomForest
model_et = ExtraTreesClassifier #ExtraTrees
model_bc = BaggingClassifier #Bagging
model_ada = AdaBoostClassifier #AdaBoost
model_dt = DecisionTreeClassifier #DecisionTree
model_xg = XGBClassifier #XGBoost
model_lr = LogisticRegression #LogisticRegression
model_svm = SVC #SVM
model_nnet = MLPClassifier # nnet

# initial parameters for each model

parameters_xg = {
        'learning_rate': [0.05,0.1],
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1],
        'subsample': [0.6, 0.8],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4],
        'n_estimators': [400],  
        'min_child_weight': [4], 
        'reg_alpha': [6,0], 
        'reg_lambda': [1,3,8],
        'max_delta_step':[2],
        'nthread':[-1], # to run at all cpu threads
        }


parameters_rf = {
    'bootstrap': [True],
    'max_depth': [ 90, 100],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 10],
    'n_estimators': [ 200, 300],
    
}

parameters_bc = {
    'n_estimators': np.arange(10, 101, 10), 
}

parameters_ada = {
              'n_estimators': [25, 50, 100, 150, 250] ,
              'learning_rate':[0.01,0.025,0.05,0.1],
             }


parameters_et=  {
     'n_estimators': [500,700,750],
     #'max_features': [25,50,75], 
     'criterion': ['entropy','gini'],
     'min_samples_split':[5,10,15],
     'max_depth':[50,75,100],
     'min_samples_leaf':[5,10,15],
 }

parameters_nnet=  {
 'activation': ['relu', 'logistic', 'tanh'],
 'alpha': [0.0001,0.005],
 'hidden_layer_sizes': [(50,),(25,2)],
 'learning_rate_init': [0.001,0.025,0.05],
 'max_iter': [700,750],
 'random_state': [None,1],
 'solver': ['adam','lbfgs'], #'sgd'

 }

parameters_svm = {'kernel':('linear', 'rbf'), 
              'C':(1,0.25,0.5,0.75),
              'gamma': (1,2,3,'auto'),
              'decision_function_shape':('ovo','ovr'),
              'shrinking':(True,False),
              'probability':(True,True)}

parameters_lr = {'C': np.arange(1e-05, 10, 0.2), 
                 'penalty': ('l2','l1'),
                 'warm_start':(True,False)
                }

parameters_dt = {'class_weight':[None,None], 
                  'criterion':['gini','gini'], 
                  'max_depth':[None,None],
                  'max_features':[None, None], 
                  'max_leaf_nodes':[None,None],
                  'min_impurity_split':[1e-07, 1e-07],
                  'min_samples_leaf':[1,1],
                  'min_samples_split':[2,2] ,
                  'min_weight_fraction_leaf':[0.0,0.0],
                  'presort':[False,False],
                  'random_state':[None, None],
                  'splitter':['best','best']}


def single_performance(df, model, clf, parameters, ncv, n_reps, nsplt):
    
    Sensitivity_Recall = np.zeros(n_reps)
    Specificity=  np.zeros(n_reps)
    Accuracy =  np.zeros(n_reps)

    for u in range(n_reps):

        df=df.sample(frac=1) #shuffle the data set
        train, test = train_test_split(df, test_size=1/nsplt)


        y_train = train['FlgQuotationStatus_Final']
        X_train = train.drop(['FlgQuotationStatus_Final','Customer','Creation_Date'], axis = 1)
        y_test = test['FlgQuotationStatus_Final']
        X_test = test.drop(['FlgQuotationStatus_Final','Customer','Creation_Date'], axis = 1)
        trainNames=list(X_train)

      
            
# balance the data
        smote = SMOTE(ratio='minority')
        X_train, y_train= smote.fit_sample(X_train, y_train)
                     
            
            #
            
        scaler = StandardScaler()  
            # Don't cheat - fit only on training data
        scaler.fit(X_train)  

        X_train = scaler.transform(X_train)  
            # apply same transformation to test data
        X_test = scaler.transform(X_test)  


        npX = np.array(X_train).copy()
        npy = np.array(y_train).copy()
    
        lb = preprocessing.LabelBinarizer()
        npy_binary=lb.fit_transform(npy)
            ##
            
            # cv for tuning hyper parameters
        grid = GridSearchCV(estimator = clf, param_grid = parameters, cv = ncv, 
                        scoring = 'roc_auc')
        grid.fit(npX,npy_binary.ravel()) #npy
            
            
            
        best_score=grid.best_score_
        best_params=grid.best_estimator_.get_params()

            #train model
        clf = model(**best_params)

        clf.fit(X_train,y_train)
        test = clf.predict(X_test)
        test_prob = clf.predict_proba(X_test)
            
        
        SR, SC, AC= testing_confusion_phase(y_test,test)    
    

        #record scores
        Sensitivity_Recall[u] = SR
        Specificity[u]= SC
        Accuracy[u]  = AC
        
    return Sensitivity_Recall, Specificity, Accuracy






In [29]:
import pandas as pd

greece = pd.read_csv('greece.csv')
df = pd.read_csv('outputdata.csv',encoding='latin1')
greece['dgreece']= 100*greece['Greece_price'].diff()/greece['Greece_price']
df = df.merge(greece,on = 'Quotation_Date', how = 'left')
df = df.dropna()
df['TranspCosts_per_Kg']=abs(df['TranspCosts_per_Kg'])
count_ratio = df.groupby('Customer').agg({'FlgQuotationStatus_Final': lambda x: len(x[x=='Won'])/len(x)}).rename(columns={'FlgQuotationStatus_Final':'Quotation_won_ratio'})
count_ratio.reset_index(level=0, inplace=True)
df = df.merge(count_ratio, how = 'outer', on = 'Customer')
df['Creation_Date'] = pd.to_datetime(df['Creation_Date'])
df = df.drop(['Customer_Country','Repackers','Quotation_Date'], axis = 1)
df = pd.get_dummies(df, columns = ['Cheese_Type','Segment','Volatility','AgeGroup','Ingredient_Applications'])
df_opt = (df.loc[(df.Customer == 'Frischpack GmbH')& (df.Cheese_Type_GOUDA == 1)][-1:]).copy()
df['PR_greece'] = df['SP_2']/df['Greece_price_x']
df.sort_values(['Customer', 'Creation_Date'], inplace=True)
df['date_diff'] = df.groupby(['Customer'])[['Creation_Date']].transform(lambda x: x.diff()).fillna(0)
df[['date_diff']]=df['date_diff'].dt.days
df[['dVolume','dValue','dSales_Price','dGreece_price','dPR_greece','dSP_2','dTransport']] = df.groupby(['Customer','Cheese_Type_GOUDA'], as_index= False)['Volume','Value','Sales_Price','Greece_price_x','PR_greece','SP_2','TranspCosts_per_Kg'].transform(lambda x: (x.diff()).fillna(0))

df=df.drop(['Greece_price_y'],axis=1)

# # OUTLIER
# df_outlier=df.loc[(df['Sales_Price']>=1.94)&(df['Sales_Price']<=3.93)]
# df=df_outlier

In [30]:
df['Production_Balance'] = 0
df['Sales']=0
df['month_year'] = df['Creation_Date'].dt.to_period('M')
df.loc[df['month_year'] == pd.to_datetime('2016-01').to_period('M'), 'Production_Balance'] = 350
df.loc[df['month_year'] == pd.to_datetime('2016-02').to_period('M'), 'Production_Balance'] = 850
df.loc[df['month_year'] == pd.to_datetime('2016-03').to_period('M'), 'Production_Balance'] = 2000
df.loc[df['month_year'] == pd.to_datetime('2016-04').to_period('M'), 'Production_Balance'] = 900
df.loc[df['month_year'] == pd.to_datetime('2016-05').to_period('M'), 'Production_Balance'] = 900
df.loc[df['month_year'] == pd.to_datetime('2016-06').to_period('M'), 'Production_Balance'] = 100
df.loc[df['month_year'] == pd.to_datetime('2016-07').to_period('M'), 'Production_Balance'] = 250
df.loc[df['month_year'] == pd.to_datetime('2016-08').to_period('M'), 'Production_Balance'] = -900
df.loc[df['month_year'] == pd.to_datetime('2016-09').to_period('M'), 'Production_Balance'] = -1000
df.loc[df['month_year'] == pd.to_datetime('2016-10').to_period('M'), 'Production_Balance'] = 200
df.loc[df['month_year'] == pd.to_datetime('2016-11').to_period('M'), 'Production_Balance'] = 400
df.loc[df['month_year'] == pd.to_datetime('2016-12').to_period('M'), 'Production_Balance'] = 1700
df.loc[df['month_year'] == pd.to_datetime('2017-01').to_period('M'), 'Production_Balance'] = 800
df.loc[df['month_year'] == pd.to_datetime('2017-02').to_period('M'), 'Production_Balance'] = 2200
df.loc[df['month_year'] == pd.to_datetime('2017-03').to_period('M'), 'Production_Balance'] = 1400
df.loc[df['month_year'] == pd.to_datetime('2017-04').to_period('M'), 'Production_Balance'] = -400
df.loc[df['month_year'] == pd.to_datetime('2017-05').to_period('M'), 'Production_Balance'] = -500
df.loc[df['month_year'] == pd.to_datetime('2017-06').to_period('M'), 'Production_Balance'] = -400
df.loc[df['month_year'] == pd.to_datetime('2017-07').to_period('M'), 'Production_Balance'] = -350
df.loc[df['month_year'] == pd.to_datetime('2017-08').to_period('M'), 'Production_Balance'] = 800
df.loc[df['month_year'] == pd.to_datetime('2017-09').to_period('M'), 'Production_Balance'] = 1500
df.loc[df['month_year'] == pd.to_datetime('2017-10').to_period('M'), 'Production_Balance'] = 250
df.loc[df['month_year'] == pd.to_datetime('2017-11').to_period('M'), 'Production_Balance'] = 1000
df.loc[df['month_year'] == pd.to_datetime('2017-12').to_period('M'), 'Production_Balance'] = 1600
df.loc[df['month_year'] == pd.to_datetime('2018-01').to_period('M'), 'Production_Balance'] = -700
df.loc[df['month_year'] == pd.to_datetime('2018-02').to_period('M'), 'Production_Balance'] = -100
df.loc[df['month_year'] == pd.to_datetime('2018-03').to_period('M'), 'Production_Balance'] = -850
df.loc[df['month_year'] == pd.to_datetime('2018-04').to_period('M'), 'Production_Balance'] = 300
df.loc[df['month_year'] == pd.to_datetime('2018-05').to_period('M'), 'Production_Balance'] = -1000
df.loc[df['month_year'] == pd.to_datetime('2018-06').to_period('M'), 'Production_Balance'] = -700
df.loc[df['month_year'] == pd.to_datetime('2018-07').to_period('M'), 'Production_Balance'] = -500
df.loc[df['month_year'] == pd.to_datetime('2018-08').to_period('M'), 'Production_Balance'] = -2800
df.loc[df['month_year'] == pd.to_datetime('2018-09').to_period('M'), 'Production_Balance'] = -1000
df.loc[df['month_year'] == pd.to_datetime('2018-10').to_period('M'), 'Production_Balance'] = 600
df.loc[df['month_year'] == pd.to_datetime('2018-11').to_period('M'), 'Production_Balance'] = 2000
df.loc[df['month_year'] == pd.to_datetime('2018-12').to_period('M'), 'Production_Balance'] = 2100


df.loc[df['month_year'] == pd.to_datetime('2016-01').to_period('M'), 'Sales'] = 18150.233887
df.loc[df['month_year'] == pd.to_datetime('2016-02').to_period('M'), 'Sales'] = 19924.3405
df.loc[df['month_year'] == pd.to_datetime('2016-03').to_period('M'), 'Sales'] = 22020.457165
df.loc[df['month_year'] == pd.to_datetime('2016-04').to_period('M'), 'Sales'] = 21589.246198
df.loc[df['month_year'] == pd.to_datetime('2016-05').to_period('M'), 'Sales'] = 21706.505662
df.loc[df['month_year'] == pd.to_datetime('2016-06').to_period('M'), 'Sales'] = 21149.638548
df.loc[df['month_year'] == pd.to_datetime('2016-07').to_period('M'), 'Sales'] = 18265.543153
df.loc[df['month_year'] == pd.to_datetime('2016-08').to_period('M'), 'Sales'] = 19766.190484
df.loc[df['month_year'] == pd.to_datetime('2016-09').to_period('M'), 'Sales'] = 20048.347281
df.loc[df['month_year'] == pd.to_datetime('2016-10').to_period('M'), 'Sales'] = 18063.732871
df.loc[df['month_year'] == pd.to_datetime('2016-11').to_period('M'), 'Sales'] = 18144.777343
df.loc[df['month_year'] == pd.to_datetime('2016-12').to_period('M'), 'Sales'] = 16037.076585
df.loc[df['month_year'] == pd.to_datetime('2017-01').to_period('M'), 'Sales'] = 18244
df.loc[df['month_year'] == pd.to_datetime('2017-02').to_period('M'), 'Sales'] = 16738
df.loc[df['month_year'] == pd.to_datetime('2017-03').to_period('M'), 'Sales'] = 20334
df.loc[df['month_year'] == pd.to_datetime('2017-04').to_period('M'), 'Sales'] = 19096
df.loc[df['month_year'] == pd.to_datetime('2017-05').to_period('M'), 'Sales'] = 20945
df.loc[df['month_year'] == pd.to_datetime('2017-06').to_period('M'), 'Sales'] = 19724
df.loc[df['month_year'] == pd.to_datetime('2017-07').to_period('M'), 'Sales'] = 16010
df.loc[df['month_year'] == pd.to_datetime('2017-08').to_period('M'), 'Sales'] = 17632
df.loc[df['month_year'] == pd.to_datetime('2017-09').to_period('M'), 'Sales'] = 16388
df.loc[df['month_year'] == pd.to_datetime('2017-10').to_period('M'), 'Sales'] = 16415
df.loc[df['month_year'] == pd.to_datetime('2017-11').to_period('M'), 'Sales'] = 18058
df.loc[df['month_year'] == pd.to_datetime('2017-12').to_period('M'), 'Sales'] = 16053
df.loc[df['month_year'] == pd.to_datetime('2018-01').to_period('M'), 'Sales'] = 19007.721823
df.loc[df['month_year'] == pd.to_datetime('2018-02').to_period('M'), 'Sales'] = 19181.119919
df.loc[df['month_year'] == pd.to_datetime('2018-03').to_period('M'), 'Sales'] = 19787.444863
df.loc[df['month_year'] == pd.to_datetime('2018-04').to_period('M'), 'Sales'] = 16884.283094
df.loc[df['month_year'] == pd.to_datetime('2018-05').to_period('M'), 'Sales'] = 19501.825713
df.loc[df['month_year'] == pd.to_datetime('2018-06').to_period('M'), 'Sales'] = 18237.525663
df.loc[df['month_year'] == pd.to_datetime('2018-07').to_period('M'), 'Sales'] = 16935.858738
df.loc[df['month_year'] == pd.to_datetime('2018-08').to_period('M'), 'Sales'] = 20871.050881
df.loc[df['month_year'] == pd.to_datetime('2018-09').to_period('M'), 'Sales'] = 18716.998901
df.loc[df['month_year'] == pd.to_datetime('2018-10').to_period('M'), 'Sales'] = 18376.869341
df.loc[df['month_year'] == pd.to_datetime('2018-11').to_period('M'), 'Sales'] = 19000
df.loc[df['month_year'] == pd.to_datetime('2018-12').to_period('M'), 'Sales'] = 19300

df = df.drop(['month_year'],axis=1)



In [31]:
#list(df)
#df=df.drop(['AgeGroup_>2','AgeGroup_0-2','Cheese_Type_GOUDA','Cheese_Type_EDAM','Ingredient_Applications_0','Ingredient_Applications_1'], axis = 1)

# positive 1 correlation
df=df.drop(['SP_2','dgreece_y', 'dSP_2','Segment_Ingredient Applications'], axis = 1)
# negative (-1) correlation
df=df.drop(['Cheese_Type_EDAM','Volatility_High','AgeGroup_>2','Ingredient_Applications_0'], axis = 1)

df=df.drop(['date_diff',
 'dVolume',
 'dValue',
 'dSales_Price',
 'dGreece_price',
 'dPR_greece',
 'dTransport','dgreece_x'], axis = 1)

df=df.drop(['Cheese_Type_GOUDA',
            'Ingredient_Applications_1',
            'Segment_Repackers',
            'Volatility_Low',
            'Segment_Industrial Food Applications',
            'CustomerAge',
            'AgeGroup_0-2',
            'Segment_0',
            'Segment_Resellers/C&C/Wholesale',
           'STDEVAVG',
           'Production_Balance',
           'Sales',
           'Greece_price_x',
           'PR_greece'], axis=1)

# y=df['FlgQuotationStatus_Final']
# df =df.drop(['FlgQuotationStatus_Final','Customer','Creation_Date'], axis = 1)
# X=df.values

In [32]:
list(df)

['Customer',
 'TranspCosts_per_Kg',
 'Creation_Date',
 'Volume',
 'Value',
 'Sales_Price',
 'FlgQuotationStatus_Final',
 'Quotation_won_ratio']

# Test a single classifier 

## rf: RandomForestClassifier

## et: ExtraTreesClassifier

## bc: BaggingClassifier

## ada: AdaBoostClassifier

## dt: DecisionTreeClassifier

## xg: XGBClassifier

## lr: LogisticRegression

## svm: SVC

## nnet: MLPClassifier


### EXAMPLE: clf=clf_bc, model= model_bc, parameters= parameters_bc

In [35]:
# Bagging classifier

ncv=4
n_reps=75
nsplt=4
model=model_bc
clf=clf_bc
parameters = parameters_bc

Sensitivity_Recall, Specificity, Accuracy= single_performance(df, model, clf, parameters,
                                                               ncv, n_reps, nsplt)

print('The average Sensitivity/Recall over ', n_reps ,' repetitions is: ', np.mean(Sensitivity_Recall),'±',np.var(Sensitivity_Recall))
print('The average Specificity over ', n_reps ,' repetitions is: ', np.mean(Specificity),'±',np.var(Specificity))
print('The average Accuracy over', n_reps ,' repetitions is: ', np.mean(Accuracy),'±',np.var(Accuracy))

The average Sensitivity/Recall over  75  repetitions is:  0.938713586231 ± 0.000351258999275
The average Specificity over  75  repetitions is:  0.837026331062 ± 0.00200639833899
The average Accuracy over 75  repetitions is:  0.912876052948 ± 0.000223238787015


# Ensemble 

In [13]:
# Ensemble


def ensemble_performance(df, models, clfs, parameters, 
                         ncv, n_reps, nsplt,out_model, 
                         out_clf, out_parameters):
    
    Sensitivity_Recall = np.zeros(n_reps)
    Specificity=  np.zeros(n_reps)
    Accuracy =  np.zeros(n_reps)

    for u in range(n_reps):

            df=df.sample(frac=1) #shuffle the data set
            train, test = train_test_split(df, test_size=1/nsplt)


            y_train = train['FlgQuotationStatus_Final']
            X_train = train.drop(['FlgQuotationStatus_Final','Customer','Creation_Date'], axis = 1)
            y_test = test['FlgQuotationStatus_Final']
            X_test = test.drop(['FlgQuotationStatus_Final','Customer','Creation_Date'], axis = 1)
            trainNames=list(X_train)

      
            
# balance the data
            smote = SMOTE(ratio='minority')
            X_train, y_train= smote.fit_sample(X_train, y_train)
                     
            
            #
            
            scaler = StandardScaler()  
            # Don't cheat - fit only on training data
            scaler.fit(X_train)  

            X_train = scaler.transform(X_train)  
            # apply same transformation to test data
            X_test = scaler.transform(X_test)  


            npX = np.array(X_train).copy()
            npy = np.array(y_train).copy()
    
            lb = preprocessing.LabelBinarizer()
            npy_binary=lb.fit_transform(npy)
            ##
            
            best_params=[]
            models_1=[]
            
            for i in range(len(models)):
                    # cv for tuning hyper parameters
                    grid = GridSearchCV(estimator = clfs[i], param_grid = parameters[i], cv = ncv, 
                        scoring = 'roc_auc')
                    grid.fit(npX,npy_binary.ravel()) #npy
            
            
            
                    #best_score=grid.best_score_
                    best_params.append(grid.best_estimator_.get_params())
                    
                    models_1.append(models[i](**best_params[i]))


                    

            S_train_1, S_test_1 = stacking(models_1,                   # list of models
                               X_train, y_train, X_test,   # data
                               
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               mode='oof_pred',            # mode: oof for train set, fit on full 
                                                           #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               #save_dir='.',               # save result and log in current dir 
                                                           #     (to disable saving - set to None)
                               metric=roc_auc_score,            # metric: callable log_loss
                               n_folds=10,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=False)                  # print all info


             # Run this before the Ensembles below
             # apply gridsearch on new data

            lb = preprocessing.LabelBinarizer()
            y_train_binary=lb.fit_transform(y_train)

            
            grid = GridSearchCV(estimator = out_clf, param_grid = out_parameters, cv = ncv, 
                        scoring = 'roc_auc')
            grid.fit(S_train_1,y_train_binary.ravel()) #npy        
        
            out_best_params=grid.best_estimator_.get_params()
            out = out_model(**out_best_params)



            out.fit(S_train_1,y_train)
            test = out.predict(S_test_1)
            test_prob = out.predict_proba(S_test_1)

            
        
            SR, SC, AC= testing_confusion_phase(y_test,test)    
    

        #record scores
            Sensitivity_Recall[u] = SR
            Specificity[u]= SC
            Accuracy[u]  = AC
        
    return Sensitivity_Recall, Specificity, Accuracy

In [14]:

ncv=3
n_reps=2
nsplt=4

#input models
models=[model_bc, model_lr, model_rf, model_ada, model_svm]
clfs=[clf_bc, clf_lr, clf_rf, clf_ada, clf_svm]
parameters = [parameters_bc, parameters_lr, parameters_rf, parameters_ada, parameters_svm]

#output_models

out_model=model_bc
out_clf=clf_bc
out_parameters=parameters_bc


Sensitivity_Recall, Specificity, Accuracy= ensemble_performance(df, models, clfs, parameters,
                                                               ncv, n_reps, nsplt,out_model, 
                                                               out_clf, out_parameters)

print('The average Sensitivity/Recall over ', n_reps ,' repetitions is: ', np.mean(Sensitivity_Recall),'±',np.var(Sensitivity_Recall))
print('The average Specificity over ', n_reps ,' repetitions is: ', np.mean(Specificity),'±',np.var(Specificity))
print('The average Accuracy over', n_reps ,' repetitions is: ', np.mean(Accuracy),'±',np.var(Accuracy))

The average Sensitivity/Recall over  2  repetitions is:  0.959525939177 ± 0.000296468025256
The average Specificity over  2  repetitions is:  0.801776531089 ± 2.1856389967e-05
The average Accuracy over 2  repetitions is:  0.92238267148 ± 0.000263915859714
